# The knapsack problem

In the knapsack problem, you need to pack a set of items $N = \{1, \dots, n\}$, with given values and sizes (such as weights or volumes), into a container with a maximum capacity. If the total size of the items exceeds the capacity, you can't pack them all. In that case, the problem is to choose a subset of the items of maximum total value that will fit in the container.

<img src="https://www.tommasoadamo.it/images/lez18/image.png" width="400px" />

* The 0/1 knapsack problem: we can't break items.
\begin{align}
\max \quad& \sum\limits_{i \in N} c_i x_i\\
\quad& \sum\limits_{i \in N} w_i x_i \leq W\\
\quad& x_i \in \{0,1\} & i \in N
\end{align}

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/oggetti.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import docplex.mp.model as cplex
import json

with open('oggetti.json', 'r') as file:
    dati = json.load(file)

W = dati['zaino']['capacita']
n = len(dati['oggetti'])

w = []
c = []

for oggetto in dati['oggetti']:
    c.append(oggetto['costo'])
    w.append(oggetto['peso'])

N = list(range(n))

with cplex.Model("0/1 knapsack") as mdl:
    x = mdl.binary_var_list(N, name="x")

    mdl.add_constraint(sum(w[i] * x[i] for i in N) <= W)
    
    mdl.maximize(sum(c[i] * x[i] for i in N))
        
    mdl.print_information()
    sol = mdl.solve(log_output=True)
    
    if sol == None:
        print('No solution.')
    else:
        print(sol)
        
        packed_items = [i for i in N if x[i].solution_value == 1]
        print('Packed items: ' + str(packed_items))
        tw = sum(w[i] for i in packed_items)
        print('Total weight:', tw)
        print('Total cost:', sol.objective_value)